In [36]:
#Single Machine Scheduling

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [37]:
single_mach = pd.DataFrame({"Arrival Time" : [0,0,0,0,0,0,0,0,0,0],
                            'Job Number':[1,2,3,4,5,6,7,8,9,10], 
                            'Processing Time':[8,2,2,8,8,4,6,9,5,10], 
                            'Due Date':[4,13,2,8,5,17,6,19,7,8],
                            "Completion Time" : [0,0,0,0,0,0,0,0,0,0],
                            "Tardiness" : [0,0,0,0,0,0,0,0,0,0],
                            "Lateness" : [0,0,0,0,0,0,0,0,0,0]})
single_mach

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
0,0,1,8,4,0,0,0
1,0,2,2,13,0,0,0
2,0,3,2,2,0,0,0
3,0,4,8,8,0,0,0
4,0,5,8,5,0,0,0
5,0,6,4,17,0,0,0
6,0,7,6,6,0,0,0
7,0,8,9,19,0,0,0
8,0,9,5,7,0,0,0
9,0,10,10,8,0,0,0


In [38]:
#Order according to EDD
single_mach = single_mach.sort_values('Due Date')
single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
single_mach["Lateness"] = single_mach["Completion Time"] - single_mach["Due Date"]

# Moore's Algorithm Steps 2-4

rejected_jobs = pd.DataFrame()
while single_mach[single_mach['Tardiness'] > 0].shape[0] > 0:
    single_mach_temp = pd.concat([single_mach[single_mach['Tardiness'] == 0] , 
                                  single_mach[single_mach['Tardiness'] > 0].head(1)])
    # Order according to processing time and reject job with largest one.
    single_mach_temp = single_mach_temp.sort_values('Processing Time', ascending=False)
    rejected_jobs = pd.concat([rejected_jobs, single_mach_temp.head(1)])
    print(single_mach_temp)
    print(rejected_jobs)
    
    # Remove rejected job from main dataframe
    single_mach = single_mach[~single_mach['Job Number'].isin(rejected_jobs['Job Number'])]
    
    # Order the main dataframe back to EDD
    single_mach = single_mach.sort_values('Due Date')
    
    # Re-compute completion time and tardiness
    single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
    single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
single_mach = pd.concat([single_mach, rejected_jobs]) #Join main dataframe with rejected jobs

single_mach_temp = single_mach_temp[0:0]
rejected_jobs = rejected_jobs[0:0]

# Finally re-compute completion times and tardiness
single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])

single_mach


   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
0             0           1                8         4               10   
2             0           3                2         2                2   

   Tardiness  Lateness  
0          6         6  
2          0         0  
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
0             0           1                8         4               10   

   Tardiness  Lateness  
0          6         6  
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
4             0           5                8         5               10   
2             0           3                2         2                2   

   Tardiness  Lateness  
4          5        13  
2          0         0  
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
0             0           1                8         4               10   
4             0           5                8   

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
2,0,3,2,2,2,0,0
8,0,9,5,7,7,0,22
1,0,2,2,13,9,0,36
5,0,6,4,17,13,0,36
0,0,1,8,4,21,17,6
4,0,5,8,5,29,24,13
6,0,7,6,6,35,29,18
3,0,4,8,8,43,35,29
9,0,10,10,8,53,45,39
7,0,8,9,19,62,43,43


In [39]:
import random
# create random df following this code: l = [random.randint(0,10) for i in range(5)]

random_jobs = pd.DataFrame({"Arrival Time":[40 + random.randint(10,40) for i in range(10)],
                            'Job Number' : [11,12,13,14,15,16,17,18,19,20],
                            'Processing Time':[random.randint(1,10) for i in range(10)], 
                            'Due Date':[7 + random.randint(0,25) for i in range(10)],
                             "Completion Time" : [0,0,0,0,0,0,0,0,0,0],
                             "Tardiness" : [0,0,0,0,0,0,0,0,0,0],
                             "Lateness" : [0,0,0,0,0,0,0,0,0,0]})
random_jobs = random_jobs.sort_values('Arrival Time')


random_jobs

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
9,51,20,3,24,0,0,0
3,54,14,1,30,0,0,0
4,62,15,10,14,0,0,0
5,63,16,5,10,0,0,0
7,64,18,9,28,0,0,0
1,70,12,5,14,0,0,0
8,71,19,1,10,0,0,0
2,72,13,10,13,0,0,0
0,73,11,5,24,0,0,0
6,77,17,6,7,0,0,0


In [40]:
#Optimizing for # of Tardy Jobs

#Step 1. 

pending_jobs = pd.DataFrame()
reschedule = pd.DataFrame()

while random_jobs[random_jobs['Due Date'] > 0].shape[0] > 0:   

    random_jobs = random_jobs.sort_values('Arrival Time')

    print(random_jobs.iloc[0,0])

    pending_jobs = single_mach.loc[(single_mach["Completion Time"] > random_jobs.iloc[0,0])]
    pending_jobs = pending_jobs.reset_index()


    print(random_jobs.iloc[0])

    pending_jobs.drop(["index"], axis = 1, inplace=True)

    pending_jobs["Arrival Time"] = pending_jobs.iloc[0,4]
    pending_jobs = pending_jobs.drop([0] , axis = 0)
    single_mach = single_mach[~single_mach['Job Number'].isin(pending_jobs['Job Number'])]
    
    print("Pending_jobs")
    print(pending_jobs)
    
    #Send pending jobs to random jobs
    random_jobs = random_jobs.append(pending_jobs)
    pending_jobs = pending_jobs[0:0]
    
    

    #Step 4

    print(single_mach.iloc[-1,4])

    reschedule = random_jobs.loc[(random_jobs["Arrival Time"] <= single_mach.iloc[-1,4])]

    print("Reschedule")
    print(reschedule)


    #Step 5

    reschedule = reschedule.sort_values("Due Date")


    #Step 6

    random_jobs = random_jobs[~random_jobs['Job Number'].isin(reschedule['Job Number'])]

    random_jobs = random_jobs.sort_values("Arrival Time")
    
    print("Randomjobs al terminar Step 6")
    print(random_jobs)




    #Recompute Completion Time

    single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
    single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
    single_mach["Lateness"] = single_mach["Completion Time"] - single_mach["Due Date"]

    print("Single_mach")
    print(single_mach)
    
    # Moore's Algorithm Steps 2-4
    while reschedule[reschedule['Tardiness'] > 0].shape[0] > 0:
        
        print("Entrando a un ciclo Moore")
        
        reschedule_temp = pd.concat([reschedule[reschedule['Tardiness'] == 0] , 
                                  reschedule[reschedule['Tardiness'] > 0].head(1)])
        # Order according to processing time and reject job with largest one.
        reschedule_temp = reschedule_temp.sort_values('Processing Time', ascending=False)
        rejected_jobs = pd.concat([rejected_jobs, reschedule_temp.head(1)])
        print(reschedule_temp)
        print(rejected_jobs)

        # Remove rejected job from main dataframe
        reschedule = reschedule[~reschedule['Job Number'].isin(rejected_jobs['Job Number'])]

        # Order the main dataframe back to EDD
        reschedule = reschedule.sort_values('Due Date')

        # Re-compute completion time and tardiness
        reschedule['Completion Time'] = reschedule['Processing Time'].cumsum()
        reschedule['Tardiness'] = np.maximum(0, reschedule['Completion Time'] - reschedule['Due Date'])
        
        print("Fin de un ciclo Moore")
        
    
    print("reschedule al salir de Moore")
    print(reschedule_temp)
    print("rejected_jobs al salir de Moore")
    print(rejected_jobs)
    reschedule = pd.concat([reschedule, rejected_jobs]) #Join main dataframe with rejected jobs
    
    reschedule_temp = reschedule_temp[0:0]
    rejected_jobs = rejected_jobs[0:0]
    
    #Append reschedule to single_mach

    single_mach = single_mach.append(reschedule)

    reschedule = reschedule[~reschedule['Job Number'].isin(single_mach['Job Number'])]

    
    print("Reschedule")
    print(reschedule)
    

    # Finally re-compute completion times and tardiness
    single_mach['Completion Time'] = single_mach['Processing Time'].cumsum()
    single_mach['Tardiness'] = np.maximum(0, single_mach['Completion Time'] - single_mach['Due Date'])
    single_mach


51
Arrival Time       51
Job Number         20
Processing Time     3
Due Date           24
Completion Time     0
Tardiness           0
Lateness            0
Name: 9, dtype: int64
Pending_jobs
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
1            53           8                9        19               62   

   Tardiness  Lateness  
1         43        43  
53
Reschedule
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
9            51          20                3        24                0   
1            53           8                9        19               62   

   Tardiness  Lateness  
9          0         0  
1         43        43  
Randomjobs al terminar Step 6
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
3            54          14                1        30                0   
4            62          15               10        14                0   
5            63          16          

   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
1            77          18                9        28               86   
2            77          15               10        14               96   

   Tardiness  Lateness  
1         58         0  
2         82         0  
77
Reschedule
   Arrival Time  Job Number  Processing Time  Due Date  Completion Time  \
2            72          13               10        13                0   
0            73          11                5        24                0   
6            77          17                6         7                0   
1            77          18                9        28               86   
2            77          15               10        14               96   

   Tardiness  Lateness  
2          0         0  
0          0         0  
6          0         0  
1         58         0  
2         82         0  
Randomjobs al terminar Step 6
Empty DataFrame
Columns: [Arrival Time, Job Number, Pr

In [41]:
single_mach

,Arrival Time,Job Number,Processing Time,Due Date,Completion Time,Tardiness,Lateness
2,0,3,2,2,2,0,0
8,0,9,5,7,7,0,0
1,0,2,2,13,9,0,-4
5,0,6,4,17,13,0,-4
0,0,1,8,4,21,17,17
4,0,5,8,5,29,24,24
6,0,7,6,6,35,29,29
3,0,4,8,8,43,35,35
9,0,10,10,8,53,45,45
9,51,20,3,24,56,32,32


In [42]:
#Performance Metrics following Moore´s Algorithm

f_bar = single_mach["Completion Time"].mean()
t_bar = single_mach["Processing Time"].mean()
mean_waiting_time = f_bar-t_bar

t_max = single_mach["Tardiness"].max()

average_lateness = single_mach["Lateness"].mean()

pos_count = [num for num in single_mach["Lateness"] if num >= 1] 
late_jobs = len(pos_count)

print("Mean Waiting time = "+ str(mean_waiting_time))
print("T Max = "+ str(t_max))
print("Number of Late Jobs = "+ str(late_jobs))


print("Flow Time = " + str(f_bar))
print("Average Lateness = " + str(average_lateness))

Mean Waiting time = 49.3
T Max = 103
Number of Late Jobs = 11
Flow Time = 55.15
Average Lateness = 21.7
